In [15]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving finetune_on_pregenerated.py to finetune_on_pregenerated.py


In [16]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [17]:
!pip install pytorch-transformers -q

     |████████████████████████████████| 184kB 2.9MB/s 
     |████████████████████████████████| 870kB 39.4MB/s 
     |████████████████████████████████| 1.0MB 48.7MB/s 


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp -q

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

## LM Finetuning

The LM finetuning code is an adaption to a script from the huggingface/pytorch-transformers repository:
* https://github.com/huggingface/pytorch-transformers/blob/v1.0.0/examples/lm_finetuning/finetune_on_pregenerated.py

Prepare the finetuning corpus, here shown for a test corpus "dev_corpus.txt":

    python pregenerate_training_data.py \
    --train_corpus **name_of_copus.txt** \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_prepared/ \
    --epochs_to_generate 2 --max_seq_len 256


Run actual finetuning with:

    python finetune_on_pregenerated.py \
    --pregenerated_data dev_corpus_prepared/ \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_finetuned/ \
    --epochs 2 --train_batch_size 16


In [21]:
!python pregenerate_training_data.py \
    --train_corpus transcript_corpus_healthcare.txt \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_prepared/ \
    --epochs_to_generate 2 --max_seq_len 256

100% 231508/231508 [00:00<00:00, 5915556.46B/s]
Loading Dataset: 902 lines [00:00, 5419.92 lines/s]
Epoch:   0% 0/2 [00:00<?, ?it/s]
Document:   0% 0/441 [00:00<?, ?it/s]
Document: 100% 441/441 [00:00<00:00, 8899.62it/s]
Document:   0% 0/441 [00:00<?, ?it/s]
Epoch: 100% 2/2 [00:00<00:00, 21.43it/s]


In [24]:
!python finetune_on_pregenerated.py \
    --pregenerated_data dev_corpus_prepared/ \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_finetuned/ \
    --epochs 2 --train_batch_size 16

2020-03-01 06:31:32,597: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2020-03-01 06:31:32,685: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2020-03-01 06:31:32,787: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmprel6gtt7
100% 361/361 [00:00<00:00, 424011.13B/s]
2020-03-01 06:31:32,867: copying /tmp/tmprel6gtt7 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
2020-03-01 06:31:32,867: creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250